# MTSとWMTGSの比較
- MTSとWMTGSのきれいな関数も作成する

In [1]:
import pandas as pd
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import chi2
import matplotlib.dates as mdates
import random
from sklearn.model_selection import train_test_split

In [15]:
df = pd.read_csv('../data/letter_recognition.csv', header=None)

#Aのみを判定するため，Aを１，A以外を0にした．
df[0] = df[0].apply(lambda x: 0 if x == 'A' else 1)

#Xとyを入力
X = df[range(1,17)]
y = df[0]

#バギング側の話
#ブートストラップサンプリングの個数
n = 10
seed = random.randint(0, n)

#使用する7つの変数をランダムに取得する
random.seed(2)
random_s = random.sample(list(X.columns), 7)
use_X = X[random_s]

X_train, X_test, y_train, y_test = train_test_split(use_X, y, test_size=0.2)

In [3]:
# 必要な関数の定義

# 共分散行列の逆行列
def inv_cov(Z):
    #標準化後のベクトルを入力する
    #標準化した後なので相関行列と分散共分散行列は一致する
    c = np.cov(Z.T)
    return np.linalg.pinv(c)

#マハラノビス汎距離
def cal_MD(Z, inv_C):
    '''
    Z:標準化したベクトル
    inv_C:分散共分散行列の逆行列
    '''
    MD = np.zeros(len(Z))
    for i in range(len(Z)):
        _a = np.dot(Z[i], inv_C)
        _MD = np.dot(_a, Z[i].T)
        _MD = _MD / Z.shape[1]
        MD[i] = _MD
    return MD
    
# 閾値をジニ係数が最小になるように決定する
def determine_threshold(y_true, y_proba):
    """
    input: 
        y_true: trainデータのラベルを入力
        y_proba: trainデータの異常度（縮小モデルのMD）を入力
    output: threshold
    """
    df_ = pd.DataFrame(y_true)
    df_['proba'] = y_proba
    df_ = df_.sort_values('proba').reset_index(drop=True)

    min_gini = np.inf
    threshold = 0
    for i in range(len(df_)):
        
        neg = df_.iloc[:i+1]
        pos = df_.iloc[i:]

        p_neg = sum(neg[y_true.name]) / len(neg)
        gini_neg = 1 - ( p_neg ** 2 + ( 1 - p_neg ) ** 2 )

        p_pos = sum(pos[y_true.name]) / len(pos)
        gini_pos = 1 - ( p_pos ** 2 + ( 1 - p_pos ) ** 2 )

        gini_split = (len(neg) / len(df_) * gini_neg) + (len(pos) / len(df_) * gini_pos)

        if min_gini > gini_split:
            min_gini = gini_split
            threshold = df_.iloc[i]['proba']
            threshold_idx = i

    return threshold

## MTS
1. 単位空間の作成
    1. Xを正常データのみで標準化してZを取得
    2. 正常データのみで共分散行列の逆行列Inv_Cを取得
2. 縮小単位空間作成のための変数選択
    1. 異常データ（anomaly_Z）を用いてSN比を算出
    2. 直交表(現状はL8のみ)を用いて各変数の効果ゲインを算出
    3. 効果ゲインが負の変数を削除し，縮小単位空間を作成
3. 縮小単位空間の閾値決定
    1. gini係数が最小となる閾値を算出
4. 新しいデータの予測
    1. 縮小単位空間によって新しいデータのマハラノビス距離を算出し，異常度とする．
    2. その異常度が閾値を超えたら異常と予測する．

In [4]:
# MTSを実行
def fit_MTS(X, y):
    """
    input: X, y
    output: reduced_model_scaler, reduced_model_inv_C, select_columns

    reduced_model_scaler: 縮小モデルのスケーラー
    reduced_model_inv_C: 縮小モデルの共分散行列の逆行列
    select_columns: 選択された変数
    """
    # 正常データのみを使用して標準化
    scaler = StandardScaler()
    scaler.fit(X[y == 0])
    normal_Z = scaler.transform(X[y == 0])
    anomaly_Z = scaler.transform(X[y == 1])

    # 正常データのみを使用して共分散行列を計算
    inv_C = inv_cov(normal_Z)

    # いったん飛ばす，削除の基準は？削除しない方法もあるっぽい？
        #１度目の仮のマハラノビス距離を計算
        # MD_1st = cal_MD(normal_Z, inv_C)
        # もしもマハラノビス距離が余りにも大きいサンプルがあれば任意で削除する
        # 削除後のデータを使用して標準化と共分散行列を計算

    # 異常データと直交表を用いてSN比を計算
    #L8直行表
    l8 = np.array([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
    l8 = (l8 == 1)

    # 異常データのマハラノビス距離
    anomaly_MD = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
    for i, l8_row in enumerate(l8):
        anomaly_MD[i] = cal_MD(anomaly_Z[:, l8_row], inv_C[l8_row][:,l8_row]) # 正常データのinv_Cを使う必要がある
    print(anomaly_MD)

    # SN比の算出
    sn = np.zeros(l8.shape[0])
    for idx, row in enumerate(anomaly_MD):
        sum_MD = 0
        for row_i in row:
            sum_MD += 1 / row_i
        sn[idx] = -10 * math.log10(sum_MD / len(row))
        
    # SN比を利用し，不要と思われる変数を削除する
    # 変数選択
    df_gain = pd.DataFrame(index=X.columns, columns=['効果ゲイン','残す'])
    for i, clm in enumerate(X.columns):
        gain = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]])
        df_gain.loc[df_gain.index == clm, '効果ゲイン'] = gain
        df_gain.loc[df_gain.index == clm, '残す'] = gain > 0
    # 選択された変数を保存
    select_columns = df_gain[df_gain['残す']].index
    
    # 選択された変数が1つ以下の場合の例外処理
    if len(select_columns) > 1:
        # 縮小モデルでのスケーラーと共分散行列を計算
        reduced_model_scaler = StandardScaler()
        reduced_model_scaler.fit(X[select_columns][y == 0])
        reduced_model_normal_Z = reduced_model_scaler.transform(X[select_columns][y == 0])
        reduced_model_inv_C = inv_cov(reduced_model_normal_Z)
    # 選択された変数が一つ以下の場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        select_columns = df_gain['効果ゲイン'].astype(float).idxmax()
        reduced_model_scaler = X[select_columns][y == 0].mean()
        reduced_model_inv_C = X[select_columns][y == 0].std()

    # 縮小モデルのスケーラーと共分散行列と選択した変数を出力
    return reduced_model_scaler, reduced_model_inv_C, select_columns
# 縮小モデルによってマハラノビス距離を計算する
def cal_MD_by_reduced_model(X, reduced_model_scaler, reduced_model_inv_C, select_columns):
    # select_columnsがfloatになることがある？
    if type(reduced_model_scaler) == StandardScaler:
        Z = reduced_model_scaler.transform(X[select_columns])
        MD = cal_MD(Z, reduced_model_inv_C)
    # 変数が一つしか選択されなかった場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        MD = ((X[select_columns] - reduced_model_scaler) / reduced_model_inv_C) ** 2
    return MD
def predict_MTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, threshold):
    proba = cal_MD_by_reduced_model(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns)
    pred = proba > threshold
    return proba, pred

In [5]:
reduced_model_scaler, reduced_model_inv_C, select_columns = fit_MTS(X_train, y_train)
y_proba_train = cal_MD_by_reduced_model(X_train, reduced_model_scaler, reduced_model_inv_C, select_columns)
threshold = determine_threshold(y_train, y_proba_train)
proba, pred = predict_MTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, threshold)
print(roc_auc_score(y_test.values, proba))

[[ 4.85357165  4.70440543 13.63909846 ...  2.43648471  0.49234346
   2.35276805]
 [ 6.1895112   0.96161499  5.55157793 ...  3.3485277   1.65387734
   2.14852772]
 [ 3.80726466  3.34197953  2.37698576 ...  3.9173113   0.21633928
   2.43517513]
 ...
 [ 2.3568608   2.94109136 23.65734927 ...  2.18872417  1.02120659
   0.82223985]
 [ 9.44715883 10.83044959 26.97138004 ...  3.39171349  1.70713183
   3.74255775]
 [ 6.28922078  1.56891532  9.15262627 ...  0.94783549  1.21761351
   1.28464218]]


In [6]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test.values, proba))

0.8830043006569847


In [7]:
select_columns

Int64Index([10, 14, 16, 12], dtype='int64')

## 結果がおかしい？？？→データがおかしかった！！！解決
---------

## WMTGS
1. 単位空間の作成
    1. Xを正常データのみで標準化してZを取得
    2. 正常データのみで共分散行列の逆行列Inv_Cを取得
2. 縮小単位空間作成のための変数重みづけ
    1. 異常データ（anomaly_Z）の転置行列からグラムシュミット直交化法によって直交ベクトル（gram_vec）を取得
    2. 直交ベクトル（gram_vec）を用いてSN比を算出
    3. 直交表(現状はL8のみ)を用いて各変数の効果ゲインを算出
    4. 効果ゲインが負の変数を削除する．
    5. 効果ゲインが正の変数グループにおいて効果ゲインによる重みを算出する．
3. 縮小単位空間の閾値決定
    1. その重みを付与した異常度を算出
    2. gini係数が最小となる閾値を算出
4. 新しいデータの予測
    1. 重みづけした縮小単位空間によって新しいデータのマハラノビス距離を算出し，異常度とする．
    2. その異常度が閾値を超えたら異常と予測する．


In [8]:
#グラムシュミット法によるマハラノビス汎距離
def gram_schmidt_cal_MD(Z, feature_weight=[1.0]*100):
    '''
    Z:標準化したベクトル
    '''
    gram_vec, _ = np.linalg.qr(Z)
    ips = np.diag(np.cov(gram_vec.T))
    k = gram_vec.shape[1]
    MD = np.zeros(len(Z))
    
    for i, one_gram_vec in enumerate(gram_vec):
        _MD = 0
        for q, u in enumerate(one_gram_vec):
            _MD += feature_weight[q] * (u**2 / ips[q])
        _MD = _MD / k
        MD[i] = _MD
    return MD

In [9]:
def fit_WMTGS(X, y):
    """
    input: X, y
    output: reduced_model_scaler, feature_weight, select_columns

    reduced_model_scaler: 縮小モデルのスケーラー
    feature_weight: 縮小モデルの特徴量の重み
    select_columns: 選択された変数
    """
    # 正常データのみを使用して標準化
    scaler = StandardScaler()
    scaler.fit(X[y == 0])
    normal_Z = scaler.transform(X[y == 0])
    anomaly_Z = scaler.transform(X[y == 1])

    # 異常データと直交表を用いてSN比を計算
    #L8直行表
    l8 = np.array([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
    l8 = (l8 == 1)

    # 異常データのマハラノビス距離
    anomaly_MD = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
    for i, l8_row in enumerate(l8):
        anomaly_MD[i] = gram_schmidt_cal_MD(anomaly_Z[:, l8_row]) # 直交化のタイミングは？これだと異常データを直交化してる
        # おそらくUでSN比を計算する
    print(anomaly_MD)

    # SN比の算出
    sn = np.zeros(l8.shape[0])
    for idx, row in enumerate(anomaly_MD):
        sum_MD = 0
        for row_i in row:
            sum_MD += 1 / row_i
        sn[idx] = -10 * math.log10(sum_MD / len(row))
        
    # SN比を利用し，不要と思われる変数を削除する
    # 変数選択
    df_gain = pd.DataFrame(index=X.columns, columns=['効果ゲイン'])
    for i, clm in enumerate(X.columns):
        gain = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]])
        df_gain.loc[df_gain.index == clm, '効果ゲイン'] = gain
    # 選択された変数を保存
    select_columns = df_gain[df_gain['効果ゲイン'] > 0].index
    
    
    # 選択された変数が1つ以下の場合の例外処理
    if len(select_columns) > 1:
        # 縮小モデルでのスケーラーと共分散行列を計算
        reduced_model_scaler = StandardScaler()
        reduced_model_scaler.fit(X[select_columns][y == 0])
        # 重みを保存
        feature_weight = (df_gain.loc[select_columns] / df_gain.loc[select_columns].sum()).values
    # 選択された変数が一つ以下の場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        select_columns = df_gain['効果ゲイン'].astype(float).idxmax()
        reduced_model_scaler = X[select_columns][y == 0].mean()
        feature_weight = X[select_columns][y == 0].std()


    return reduced_model_scaler, feature_weight, select_columns

# 縮小モデルによってマハラノビス距離を計算する
def gram_schmidt_cal_MD_by_reduced_model(X, reduced_model_scaler, feature_weight, select_columns):
    # select_columnsがfloatになることがある？
    if type(reduced_model_scaler) == StandardScaler:
        Z = reduced_model_scaler.transform(X[select_columns])
        MD = gram_schmidt_cal_MD(Z, feature_weight)
    # 変数が一つしか選択されなかった場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        MD = ((X[select_columns] - reduced_model_scaler) / feature_weight) ** 2
    return MD

def predict_WMTGS(X_test, reduced_model_scaler, feature_weight, select_columns, threshold):
    proba = gram_schmidt_cal_MD_by_reduced_model(X_test, reduced_model_scaler, feature_weight, select_columns)
    pred = proba > threshold
    return proba, pred

In [18]:
reduced_model_scaler, feature_weight, select_columns = fit_WMTGS(X_train, y_train)
y_proba_train = gram_schmidt_cal_MD_by_reduced_model(X_train, reduced_model_scaler, feature_weight, select_columns)
threshold = determine_threshold(y_train, y_proba_train)
proba, pred = predict_WMTGS(X_test, reduced_model_scaler, feature_weight, select_columns, threshold)

[[0.55374356 0.86808388 1.18225135 ... 1.04045303 1.46085567 0.53424671]
 [0.60913342 0.65978089 0.57324622 ... 1.4859584  1.77811602 0.25698666]
 [0.92303311 1.17351489 1.7786189  ... 1.04109025 1.19587482 0.73040287]
 ...
 [0.37729101 0.67742734 0.63723131 ... 0.75152921 1.5678866  0.10127744]
 [0.80576905 1.74328186 1.29606174 ... 1.318261   0.60235032 1.19467536]
 [1.00378495 1.47406687 0.60598906 ... 1.62831045 1.8827214  1.46878261]]


In [20]:
print(roc_auc_score(y_test.values, proba))

0.46388977827583444


In [16]:
reduced_model_scaler, feature_weight, select_columns = fit_WMTGS(X_train, y_train)
y_proba_train = gram_schmidt_cal_MD_by_reduced_model(X_train, reduced_model_scaler, feature_weight, select_columns)
threshold = determine_threshold(y_train, y_proba_train)
proba, pred = predict_WMTGS(X_test, reduced_model_scaler, feature_weight, select_columns, threshold)
print(roc_auc_score(y_test.values, proba))

[[0.55374356 0.86808388 1.18225135 ... 1.04045303 1.46085567 0.53424671]
 [0.60913342 0.65978089 0.57324622 ... 1.4859584  1.77811602 0.25698666]
 [0.92303311 1.17351489 1.7786189  ... 1.04109025 1.19587482 0.73040287]
 ...
 [0.37729101 0.67742734 0.63723131 ... 0.75152921 1.5678866  0.10127744]
 [0.80576905 1.74328186 1.29606174 ... 1.318261   0.60235032 1.19467536]
 [1.00378495 1.47406687 0.60598906 ... 1.62831045 1.8827214  1.46878261]]
0.46388977827583444


In [17]:
reduced_model_scaler, reduced_model_inv_C, select_columns = fit_MTS(X_train, y_train)
y_proba_train = cal_MD_by_reduced_model(X_train, reduced_model_scaler, reduced_model_inv_C, select_columns)
threshold = determine_threshold(y_train, y_proba_train)
proba, pred = predict_MTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, threshold)
print(roc_auc_score(y_test.values, proba))

[[ 3.43353511  3.45841872  4.66272388 ...  3.99618571  6.966797
   3.37775447]
 [ 3.34211066  2.34920689  1.67698028 ... 10.75428927  6.71097596
   1.86915639]
 [ 2.55257625  3.42176251  3.66494702 ...  1.70346274  2.33673349
   1.40839504]
 ...
 [ 1.1337538   4.22292582  1.69113612 ...  4.1120787   4.59842975
   0.48379127]
 [ 2.54233103 10.65263276  3.50780256 ... 11.3289156   1.28692469
   3.98756192]
 [ 3.72405432  8.91262925  3.56644918 ...  9.98440544  8.39368719
   8.1239551 ]]
0.884659576029243


## 課題！！！
- 異常データのマハラノビス距離を求める際に，MTSでは正常データの標準偏差，平均で標準化した後，正常データの分散共分散行列でマハラノビス距離を求める．つまり，異常データを正常データと同じ軸で扱っている．
- しかし，グラムシュミット法では，異常データで直交化を行い，さらに共分散行列を用いないため，異常データのイプシロンでマハラノビス距離を求めている．
- 正常データでマハラノビス距離を求めるときは結果は一緒になるが異常データを用いるとマハラノビス距離がやや変わってしまう？

# MTGSでは直交表使わないっぽいぞ？？？？？
### いや使うけど計算方法が違うっぽい！
### MTGSでは変数ごとにMDが求められるから直交化は全部の変数でやってMDの計算のところだけ変数を入れるか入れないか区別する！！！！
#### まあ現状のでも悪くないかもだけど

## 20220413
### とりあえずできそうなことやってみる．金曜日にわからないことまとめて月曜日に先生に質問投げる